In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [7]:
%autoreload
from pathlib import Path
import sys
root = Path().resolve().absolute().parent.parent
print(root)
sys.path.append(str(root))

from src.pipeline import Pipeline, BuilingIdsEnum
pipe = Pipeline()



/Users/matsalexander/Desktop/SolarEnergyImpact
Index(['HourUTC', 'HourDK', 'PriceArea', 'SpotPriceDKK', 'SpotPriceEUR'], dtype='object')


In [8]:
building_b = pipe.get_data(BuilingIdsEnum.B)
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

# remove from 3. to 7. july 2024 from dataset A
# mask = (building_a['timestamp'] >= '2024-07-03') & (building_a['timestamp'] <= '2024-07-07')
# building_a = building_a[~mask]
# let value_import be equal to net_consumption for main
main_building['value_import'] = main_building['net_consumption']

# normalize the value_import by area and call it value for main and building_b
main_building['value'] = main_building['value_import'] / main_building['area']
building_b['value'] = building_b['value_import'] / building_b['area']

combined_df = pd.concat([main_building, building_b])
# reset index
combined_df = combined_df.reset_index(drop=True)
# Perform the train-test split with stratification based on 'building_id'
train_data, test_data = train_test_split(
    combined_df,
    test_size=0.20,
    random_state=42,
    stratify=combined_df['building']
)

# select features
target = "value"
features = ["timestamp", "area", "temperature", "wind_speed"] #, "cloud_fraction", "precipitation"

train_data = train_data[features + [target]]
test_data = test_data[features + [target]]



In [9]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN']
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20241111_184001"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       13.34 GB / 32.00 GB (41.7%)
Disk Space Avail:   615.03 GB / 926.35 GB (66.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal 

[1000]	valid_set's l1: 0.00472896
[2000]	valid_set's l1: 0.00448904
[3000]	valid_set's l1: 0.00435077
[4000]	valid_set's l1: 0.00425569
[5000]	valid_set's l1: 0.00418858
[6000]	valid_set's l1: 0.00413702
[7000]	valid_set's l1: 0.00410055
[8000]	valid_set's l1: 0.00407165
[9000]	valid_set's l1: 0.00404406
[10000]	valid_set's l1: 0.00401983
[1000]	valid_set's l1: 0.0047699
[2000]	valid_set's l1: 0.00458817
[3000]	valid_set's l1: 0.00447966
[4000]	valid_set's l1: 0.00442139
[5000]	valid_set's l1: 0.00437038
[6000]	valid_set's l1: 0.00432504
[7000]	valid_set's l1: 0.00429198
[8000]	valid_set's l1: 0.00426502
[9000]	valid_set's l1: 0.00424478
[10000]	valid_set's l1: 0.00423059
[1000]	valid_set's l1: 0.00485891
[2000]	valid_set's l1: 0.00466093
[3000]	valid_set's l1: 0.0045501
[4000]	valid_set's l1: 0.00447003
[5000]	valid_set's l1: 0.00440727
[6000]	valid_set's l1: 0.00435942
[7000]	valid_set's l1: 0.00432405
[8000]	valid_set's l1: 0.00429692
[9000]	valid_set's l1: 0.00427844
[10000]	valid_

	-0.0042	 = Validation score   (-mean_absolute_error)
	363.99s	 = Training   runtime
	2.1s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 230.61s of the 530.61s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 0.00412601
[2000]	valid_set's l1: 0.00404992
[3000]	valid_set's l1: 0.0040444
[1000]	valid_set's l1: 0.00419994
[2000]	valid_set's l1: 0.00415827
[3000]	valid_set's l1: 0.00414751
[1000]	valid_set's l1: 0.00436727
[2000]	valid_set's l1: 0.00426687
[3000]	valid_set's l1: 0.00425386
[4000]	valid_set's l1: 0.00424554
[1000]	valid_set's l1: 0.00414138
[2000]	valid_set's l1: 0.00404368
[3000]	valid_set's l1: 0.00399897
[4000]	valid_set's l1: 0.00399908
[1000]	valid_set's l1: 0.00427519
[2000]	valid_set's l1: 0.00422783
[3000]	valid_set's l1: 0.00421783
[4000]	valid_set's l1: 0.0042188
[1000]	valid_set's l1: 0.00427314
[2000]	valid_set's l1: 0.00418951
[3000]	valid_set's l1: 0.00417945
[1000]	valid_set's l1: 0.00439342
[2000]	valid_set's l1: 0.00430359
[3000]	valid_set's l1: 0.00428648
[1000]	valid_set's l1: 0.00426344
[2000]	valid_set's l1: 0.00418495
[3000]	valid_set's l1: 0.00416015
[4000]	valid_set's l1: 0.00415603
[5000]	valid_set's l1: 0.00415085
[6000]	valid_set

	-0.0042	 = Validation score   (-mean_absolute_error)
	139.69s	 = Training   runtime
	0.49s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 89.83s of the 389.84s of remaining time.
	-0.0034	 = Validation score   (-mean_absolute_error)
	2.14s	 = Training   runtime
	0.47s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 86.93s of the 386.94s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 2585.
	Ran out of time, early stopping on iteration 2666.
	Ran out of time, early stopping on iteration 2788.
	Ran out of time, early stopping on iteration 2865.
	Ran out of time, early stopping on iteration 2933.
	Ran out of time, early stopping on iteration 3087.
	Ran out of time, early stopping on iteration 3269.
	Ran out of time, early stopping on iteration 3659.
	-0.0043	 = Validation score   (-mean_absolute_error)
	83.36s

[1000]	valid_set's l1: 0.00482935
[2000]	valid_set's l1: 0.00461036
[3000]	valid_set's l1: 0.0044795
[4000]	valid_set's l1: 0.00439591
[5000]	valid_set's l1: 0.00433442
[6000]	valid_set's l1: 0.00427695
[7000]	valid_set's l1: 0.0042304
[8000]	valid_set's l1: 0.00419964
[9000]	valid_set's l1: 0.00417641
[10000]	valid_set's l1: 0.00416157
[1000]	valid_set's l1: 0.00492026
[2000]	valid_set's l1: 0.00469514
[3000]	valid_set's l1: 0.00457025
[4000]	valid_set's l1: 0.00448159
[5000]	valid_set's l1: 0.00442187
[6000]	valid_set's l1: 0.00438088
[7000]	valid_set's l1: 0.00434411
[8000]	valid_set's l1: 0.00431559
[9000]	valid_set's l1: 0.00429465
[10000]	valid_set's l1: 0.00427878
[1000]	valid_set's l1: 0.00481017
[2000]	valid_set's l1: 0.0045881
[3000]	valid_set's l1: 0.00445241
[4000]	valid_set's l1: 0.00435743
[5000]	valid_set's l1: 0.00429681
[6000]	valid_set's l1: 0.0042522
[7000]	valid_set's l1: 0.00422262
[8000]	valid_set's l1: 0.00418963
[9000]	valid_set's l1: 0.00416986
[10000]	valid_se

	-0.0042	 = Validation score   (-mean_absolute_error)
	551.78s	 = Training   runtime
	2.72s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 1242.41s of the 2142.53s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 0.00425082
[2000]	valid_set's l1: 0.00415555
[3000]	valid_set's l1: 0.00412293
[4000]	valid_set's l1: 0.0041114
[5000]	valid_set's l1: 0.00411928
[1000]	valid_set's l1: 0.00425988
[2000]	valid_set's l1: 0.00414088
[3000]	valid_set's l1: 0.00409826
[4000]	valid_set's l1: 0.00408468
[5000]	valid_set's l1: 0.00408067
[1000]	valid_set's l1: 0.00420067
[2000]	valid_set's l1: 0.00411525
[3000]	valid_set's l1: 0.00408929
[1000]	valid_set's l1: 0.00415359
[2000]	valid_set's l1: 0.00405715
[3000]	valid_set's l1: 0.00404306
[1000]	valid_set's l1: 0.00427508
[2000]	valid_set's l1: 0.00423254
[3000]	valid_set's l1: 0.00422948
[1000]	valid_set's l1: 0.00421013
[2000]	valid_set's l1: 0.00416374
[3000]	valid_set's l1: 0.00414688
[4000]	valid_set's l1: 0.00414682
[1000]	valid_set's l1: 0.0041934
[2000]	valid_set's l1: 0.00408812
[3000]	valid_set's l1: 0.00406765
[1000]	valid_set's l1: 0.00409936
[2000]	valid_set's l1: 0.0040537
[3000]	valid_set's l1: 0.00404045


	-0.0041	 = Validation score   (-mean_absolute_error)
	258.01s	 = Training   runtime
	0.53s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 983.2s of the 1883.32s of remaining time.
	-0.0033	 = Validation score   (-mean_absolute_error)
	2.26s	 = Training   runtime
	0.57s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 980.11s of the 1880.23s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 5185.
	Ran out of time, early stopping on iteration 3130.
	Time limit exceeded... Skipping CatBoost_BAG_L1.
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 340.87s of remaining time.
	Ensemble Weights: {'RandomForestMSE_BAG_L1': 1.0}
	-0.0033	 = Validation score   (-mean_absolute_error)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
Excluded models: ['KNN'] (Specified by `excluded_mode

In [10]:
# AutogluonModels/ag-20241109_185606
# predictor = TabularPredictor.load("AutogluonModels/ag-20241109_185606")
# Evaluate on test data
performance = predictor.evaluate(test_data)
# best model: ag-20241022_161331

print("Evaluation Performance:")
performance
# reset index 
test_data = test_data.reset_index(drop=True)
# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions


Computing feature importance via permutation shuffling for 4 features using 4142 rows with 5 shuffle sets...


Evaluation Performance:


	6.79s	= Expected runtime (1.36s per shuffle set)
	2.09s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
             importance    stddev       p_value  n  p99_high   p99_low
timestamp      0.003790  0.000075  1.841552e-08  5  0.003945  0.003636
temperature    0.003285  0.000047  4.952707e-09  5  0.003381  0.003188
area           0.002735  0.000068  4.589780e-08  5  0.002876  0.002595
wind_speed     0.000635  0.000032  7.981253e-07  5  0.000702  0.000569


In [15]:
performance

{'mean_absolute_error': -0.0033329320087102667,
 'root_mean_squared_error': -0.004850775099255721,
 'mean_squared_error': -2.353001906355935e-05,
 'r2': 0.7085213089279626,
 'pearsonr': 0.8420261238965251,
 'median_absolute_error': -0.00227640745880669}

In [11]:
# model location => AutogluonModels/ag-20241016_095906
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

data_predict = main_building[features]
data_predict


,timestamp,area,temperature,wind_speed
0,2023-07-01 00:00:00,1199,13.6,1.6
1,2023-07-01 01:00:00,1199,13.2,2.0
2,2023-07-01 02:00:00,1199,12.3,1.6
3,2023-07-01 03:00:00,1199,11.9,0.6
4,2023-07-01 04:00:00,1199,11.9,0.2
...,...,...,...,...
10338,2024-09-03 18:00:00,1199,17.8,2.5
10339,2024-09-03 19:00:00,1199,17.8,1.8
10340,2024-09-03 20:00:00,1199,17.7,1.1
10341,2024-09-03 21:00:00,1199,18.0,3.4


In [12]:
prediciton1 = predictor.predict(data_predict)

In [ ]:
# save predicitons as a csv in data folder from root.
import datetime
from pathlib import Path


prediciton1_df = pd.DataFrame(prediciton1)
date_time = datetime.datetime.now().strftime("%Y%m%d")
my_path = Path().resolve().parent.parent / 'data'/ "pred" / 'prediction_main_b.csv'
# create folder
my_path.parent.mkdir(parents=True, exist_ok=True)
prediciton1_df.to_csv(my_path, index=False)